In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql
import dash_bootstrap_components

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots


ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"

%load_ext autoreload
%autoreload 2

conn = hk_psql.connect_psql('mimic')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


DATABASE CONNECTED !!!


# load a cohort

In [2]:
# load a cohort
cohort_name = 'sepsis'



schema_core = cohort_name+'_mimic_core'
df_overview = hk_psql.summary(schema_core, conn)
hadm_ids = df_overview['hadm_id'].unique()

q=f"""
    select * from mimic_core.patients t1
    inner join {schema_core}.admissions t2
    on t1.subject_id = t2.subject_id
    where t2.hadm_id in ({str(hadm_ids.tolist())[1:-1]})
    
    """
# print(q)

df_patients = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.transfers         
        """
# print(q)

df_transfers = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.admissions         
        """
# print(q)

df_admissions = hk_psql.query(q, conn)

q=f"""
        select * from {cohort_name+'_mimic_icu'}.icustays         
        """
# print(q)

df_icustay = hk_psql.query(q, conn)

# LAB EVENTS
q=f"""
select * from {cohort_name+'_mimic_hosp'}.labevents         
"""

df_le = hk_psql.query(q, conn)

# icd_procedure code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.procedures_icd         
"""
df_icd_proc = hk_psql.query(q, conn)

# icd_diagnoses code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.diagnoses_icd         
"""
df_icd_diag = hk_psql.query(q, conn)


# CHART EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.chartevents         
"""

df_che = hk_psql.query(q, conn)

# PROCEDURE EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.procedureevents         
"""

df_pe = hk_psql.query(q, conn)

# Medications
q=f"""
select * from {cohort_name+'_mimic_hosp'}.emar         
"""

df_emar = hk_psql.query(q, conn)

df_patients = df_patients.loc[:,~df_patients.columns.duplicated()]
print(df_patients['subject_id'])
print(df_transfers['subject_id'])


cols_to_use = [x for x in df_transfers.columns.tolist() if x not in df_patients.columns.tolist()]
cols_to_use.append('subject_id')
print(cols_to_use)
df_demo = pd.merge(df_patients, df_transfers[cols_to_use], how='inner',  on='subject_id')

print(df_demo)
df_le_main = df_le.copy()

--- 0.023723840713500977 seconds ---
--- 0.0953221321105957 seconds ---
--- 0.055361032485961914 seconds ---
--- 0.01658344268798828 seconds ---
--- 0.021076679229736328 seconds ---
--- 20.612441301345825 seconds ---
--- 0.06486344337463379 seconds ---
--- 0.1501603126525879 seconds ---
--- 235.47201585769653 seconds ---
--- 0.8744573593139648 seconds ---
--- 10.842583656311035 seconds ---
0       10545747
1       12823483
2       14122388
3       14122388
4       14819767
          ...   
2805    19793638
2806    19962418
2807    19836795
2808    19921864
2809    19990427
Name: subject_id, Length: 2810, dtype: int64
0        12567919
1        13758211
2        18109635
3        14371035
4        16151261
           ...   
15664    16800514
15665    19439124
15666    17851873
15667    17513048
15668    10352268
Name: subject_id, Length: 15669, dtype: int64
['transfer_id', 'eventtype', 'careunit', 'intime', 'outtime', 'subject_id']
       subject_id gender  anchor_age  anchor_year ancho

In [131]:
q = df_transfers['careunit'].str.contains('MICU', na=False)
q
# df_transfers.loc[q,'careunit'].unique().tolist()
df_los = (df_transfers.loc[q, 'outtime'] - df_transfers.loc[q, 'intime']) / np.timedelta64(1,'h')
df_los.describe()

0        False
1        False
2        False
3        False
4        False
         ...  
15664    False
15665    False
15666    False
15667    False
15668    False
Name: careunit, Length: 15669, dtype: bool

count    2966.000000
mean      101.437379
std       131.405869
min         0.002500
25%        22.499097
50%        58.620972
75%       123.872431
max      1802.381389
dtype: float64

# vis pat

## select vars

In [105]:
# step 1) find the patient based on criteria
# use [VI] to find the transfer id
######################################## select variables
 
dict_vitals = {
    220045:['HR', [60,100], ['L', 'N', 'H'] ],
    220210:['RR', [10,20], ['L', 'N', 'H'] ],
    220277:['SpO2', [90,101], ['L', 'N'] ],
}


dict_labs = {
    50971:['Potassium',[3.3,5],['L', 'N', 'H']], 
    50983:['Sodium',[133,145],['L', 'N', 'H']], 
    50902:['Chloride',[96,108],['L', 'N', 'H']], 
    # 51006:['Urea Nitrogen',[6,20],['L', 'N', 'H']], 
    # 50912:['Creatinine',[0.5,1.2],['L', 'N', 'H']], 
    # 50889:['CRP',[0, 5],['L', 'N', 'H']], 
    # 51652:['HS CRP',[0, 5],['L', 'N', 'H']], 


    # 51301:['WBC',[4,10],['L', 'N', 'H']], 
    # 51279:['RBC',[4.6,6.2],['L', 'N', 'H']], 
    # 50802:['BE',[-2,2],['L', 'N', 'H']], 
    # 50882:['HCO3',[22,32],['L', 'N', 'H']], 
    50820:['pH',[7.35,7.45],['L', 'N', 'H']], 



    51003:[	'Troponin T',[0,0.1],['L', 'N', 'H']],
    #50910:[	'Creatine Kinase (CK)'],
    50813:[	'Lactate',[0.5,2],['L', 'N', 'H']],
}
# dict_labs = {

#     50813:[	'Lactate',[0.5,2],['L', 'N', 'H']],
# }

list_emar = ['NORepinephrine','EPINEPHrine','DOBUTamine','DOPamine','PHENYLEPHrine']
dict_proc = {225792:'Invasive Ventilation', 225802:'Dialysis - CRRT', 220277:'SpO2'}


In [106]:
# function to extract the data

def f_extract(hid, t_in, t_out):

    # Extracting vitals from ICU chartevents
    dff_che = df_che[df_che['hadm_id']==hid].query(f"itemid.isin({list(dict_vitals.keys())})").merge(pd.read_csv(ADD_DATA+'d_items.csv')[['itemid','label']], on='itemid')
    dff_che = dff_che[['charttime','itemid','valuenum','label']].sort_values('charttime',ascending=True) 
    dff_che = dff_che.rename(columns={'valuenum':'value'})
    dff_che['type'] = 'vital'
      
    dff_che.drop(dff_che.index[dff_che['charttime']> t_out], inplace=True)
    dff_che.drop(dff_che.index[dff_che['charttime']< t_in], inplace=True)

    dff_che['time_grid'] = ( (dff_che['charttime'] - t_in) / np.timedelta64(1, 'h')).astype(float)
    # dff_che = dff_che[dff_che['time_grid']>0]
    dff_che = dff_che.drop_duplicates(subset=['itemid','time_grid'],keep='first')
    dff_che = dff_che.sort_values(['itemid','charttime'])
    ## creating mask for each vital
    dff_che['mask'] = np.nan
    for itemid, values in dict_vitals.items():
        q = dff_che['itemid']==itemid    
    

        if not q.empty:
            dff_che.loc[q, 'mask'] = pd.cut(dff_che.loc[q,'value'], bins=[-1e10,*values[1],1e10], labels=['L','N','H']).astype(str)
            dff_che.loc[q, 'change'] = dff_che.loc[q, 'mask'].shift(1) + dff_che.loc[q, 'mask']
            dff_che.loc[q, 'lb'] = values[1][0]
            dff_che.loc[q, 'ub'] = values[1][1]
        

    # print(dff_che)


    # Extracting labs from HOSP labevents
    dff_le = df_le[df_le['hadm_id']==hid].query(f"itemid.isin({list(dict_labs.keys())})").merge(pd.read_csv(ADD_DATA+'d_labitems.csv')[['itemid','label']], on='itemid')
    dff_le = dff_le[['charttime','itemid','value','label']].sort_values('charttime',ascending=True)
    dff_le['type'] = 'lab'
    dff_le.drop(dff_le.index[dff_le['charttime']> t_out], inplace=True)
    dff_le.drop(dff_le.index[dff_le['charttime']< t_in], inplace=True)

    dff_le['time_grid'] = ( (dff_le['charttime'] - t_in) / np.timedelta64(1, 'h')).astype(float)
    # dff_le = dff_le[dff_le['time_grid']>0]
    dff_le = dff_le.drop_duplicates(subset=['itemid','time_grid'],keep='first')
    dff_le = dff_le.sort_values(['itemid','charttime'])

    ## creating mask for each lab
    dff_le['mask'] = np.nan
    for itemid, values in dict_labs.items():
        # print(values)
        q = dff_le['itemid']==itemid
        if not q.empty:
            # print(values)
            # print(dff_le.loc[q,'value'])
            # print(dff_le.loc[q,'value'].values)
            # print(dff_le.loc[q,'value'].astype(float))
            
            dff_le.loc[q,'value']
            dff_le.loc[q, 'mask'] = pd.cut(dff_le.loc[q,'value'].astype(float), bins=[-1e10,*values[1],1e10], labels=['L','N','H']).astype(str)
            # print(dff_le[q])
            # print(dff_le.dtypes)
            # print(dff_le[q]['mask'].shift(1, fill_value='F'))
            # print(dff_le.loc[q, 'mask'])

            dff_le.loc[q, 'change'] = dff_le.loc[q, 'mask'].shift(1, fill_value='F') + dff_le.loc[q, 'mask']
            # print(dff_le.loc[q])
            # print(dff_le.loc[q, 'change'])
            # print(dff_le)
            
            dff_le.loc[q, 'lb'] = values[1][0]
            dff_le.loc[q, 'ub'] = values[1][1]
            # print(dff_le)


    dff_le
    # print(dff_le)
    # term

    dff_le_samp = dff_le.copy()
    dff_le_samp['type'] = 'lab_samp'
    dff_le_samp['mask'] = np.nan
    dff_le_samp['label'] = dff_le_samp['label']+'_sam'
    dff_le_samp['label2'] = dff_le_samp['label']

    dff_emar = df_emar[df_emar['hadm_id']==hid].query(f"medication.isin({list_emar})")
    dff_emar = dff_emar[['charttime','medication']].sort_values('charttime',ascending=True)
    
    dff_emar.drop(dff_emar.index[dff_emar['charttime']> t_out], inplace=True)

    dff_emar['value'] = 0
    dff_emar['value2'] = 1
    dff_emar['lb'] = 0
    dff_emar['ub'] = 0
    dff_emar['mask'] = 'Treatment'
    dff_emar['label2'] = dff_emar['medication']
    dff_emar['type'] = 'med'
    dff_emar['time_grid'] = ( (dff_emar['charttime'] - t_in) / np.timedelta64(1, 'h')).astype(float)
    dff_emar = dff_emar[dff_emar['time_grid']>0]
    dff_emar = dff_emar.rename(columns={'medication':'label'})
    dff_emar

    df_con = pd.concat([dff_che,dff_le,dff_emar,dff_le_samp],ignore_index=True) # ignore_index is necessary
    
    

    return df_con

    

## reading data

In [3]:
# step 2) visualize 
from IPython.display import display

tids = [39658297, 35098730, 39855901, 39663204, 33298382, 31037601, 34014186] # to be extracted from GUI
tids = [31037601, 33298382, 31093400, 30803697, 35119398, 34250110, 38720996, 35768370, 36400963, 37448135, 33567966, 38538012, 31271824, 33592773, 32357993, 36764861, 37423812, 35185510, 36295684, 30495523, 33385967, 39737923, 34980706, 30980315, 35375459, 34843071, 34773746, 37815113, 34369768, 33171787, 33344655, 33759916, 39203865, 32793644, 38713313, 31857748, 32332328, 32809464, 31520383, 34187423, 32892240, 37570047, 35795842]
tids = [34187423,32793644,31857748]
# q = df_transfers['careunit'].str.contains('MICU', na=False)
# tids = df_transfers.loc[q,'transfer_id'].unique().tolist()
# print(len(tids))

# tids = [38236111, 37003426, 31344039, 31103970, 34703950, 39028259, 35196506, 35141988, 37030246, 34250110, 39064788, 39165313, 34992538, 31037601, 30378500, 33882599, 34586400, 32414218, 31570655, 31520383, 33817343, 36192138, 33567775, 38102028, 34729434, 32774203, 33298382, 36779265, 36260187, 37493191, 36324036, 35452862, 34673371, 34687736, 33436208, 37278476, 36821110, 38825792, 35167591, 32206072, 35007084, 31688300, 31522155, 30336275, 38706653, 36764861, 31086094, 34369768, 37182448, 33567966, 31417132, 32852594, 32181960, 32589838, 38720996, 38622850, 34728659, 32735812, 36488865, 32236052, 38726679, 38914211, 31577045, 38736555, 30814401, 32983756, 31281842, 31920615, 39272262, 36298131, 35527300, 32332328, 32317492, 31276436, 33987172, 31945426, 39774176, 38948241, 30980315, 39286797, 37124281, 37734261, 35768370, 34935331, 33495934, 30447229, 39470226, 34654823, 38127185, 32480681, 39855901, 30787940, 31857748, 38426371, 35375459, 38904914, 34169848, 37585407, 36400963, 34883463, 31005953, 34943069, 36374245, 39888479, 31274907, 33418928, 34573429, 34608866, 39496285, 33338599, 39574750, 30188845, 31312914, 37081846, 34843071, 35770004, 34511426, 30065590, 30733669, 31093630, 34530113, 35356603, 32334402, 36841282, 36158899, 39816989, 35589556, 38713313, 37620547, 31034556, 36295684, 35808931, 31052418, 32939825, 34809083, 35573393, 33684823, 32910659, 32528359, 34198359, 35462957, 39194887, 30089953, 38197697, 36633379, 34638150, 33498463, 32964078, 38884238, 36834095, 39867336, 34698001, 30304561, 35482478, 35850856, 39768852, 39555696, 39663204, 38599401, 38882770, 36637825, 32856945, 35739022, 39496225, 39073737, 35213955, 37783673, 34773746, 31172345, 30009597, 35311785, 38632749, 36126902, 36422887, 35688748, 35609937, 32530604, 35819403, 30826769, 37448135, 39658297, 31548284, 39796559, 34133437, 39120208, 31068797, 31085333, 38264024, 37494022, 39522082, 32237762, 32256558, 37334760, 39099519, 37683695, 36237605, 30114664, 33769962, 34798760, 39233063, 33551572, 34149913, 38349238, 38584436, 33806692, 38793520, 35795842, 30851647, 30495523, 38497808, 38108122, 31300090, 39899794, 32883572, 32216705, 38553829, 39648340, 37403074, 31550648, 37451901, 32573088, 35602103, 33344655, 34014186, 37432009, 36430031, 35922901, 30507903, 39036751, 31275587, 35873429, 38692998, 34402278, 33639629, 38432710, 30033143, 34402613, 31567309, 39474343, 34033580, 30449036, 38213244, 31294814, 39936128, 32123185, 36288131, 34785841, 37004565, 31777308, 34731397, 36040765, 33804332, 39635063, 35630886, 31444117, 36302606, 33363365, 37038952, 34187423, 34897008, 39709810, 32030117, 31427340, 35479676, 32731163, 35682633, 33115332, 39140365, 34218332, 30803697, 34689947, 30333109, 36033368, 38019228, 37207819, 35411377, 32735632, 32831029, 37622645, 32664351, 38536103, 38070327, 36161971, 31792987, 34400144, 33171787, 37501442, 31271824, 38519468, 32384467, 36120303, 32763512, 37082589, 33112399, 36519825, 31276987, 34005385, 31485023, 36607247, 30832161, 32398002, 32850677, 38312172, 39837616, 33655764, 33373105, 35664828, 36362872, 32068796, 35811167, 39020494, 35014088, 35153487, 30963809, 30842264, 36533146, 35988527, 36269947, 31658487, 31025939, 38954915, 33404486, 31875839, 37921269, 33648412, 31732778, 38102168, 39934716, 31681061, 31093400, 39620145, 38367142, 37146017, 37295515, 37284087, 35876442, 34755431, 32892240, 36294083, 36732527, 37211192, 32709585, 38556021, 37268115, 33385967, 37056137, 36582870, 38330487, 30173232, 39247539, 30158243, 34918996, 35541401, 32198117, 33216558, 33773468, 31168390, 38792371, 32380994, 31330463, 32062515, 35306693, 32229263, 36754410, 32402030, 32379334, 32748678, 36032422, 31950417, 30409527, 39723055, 37298868, 37807704, 36763879, 31299833, 39327465, 37423812, 36617225, 34233623, 34034392, 36788229, 37575309, 37435891, 36838254, 30039090, 30652743, 30413290, 33296875, 37607625, 38549794, 36809832, 34292315, 33817347, 31056009, 32567149, 32043259, 38821385, 39158918, 33948578, 31321259, 34259210, 37312467, 34995687, 38320976, 37364163, 36726770, 37373364, 37813117, 31872089, 33393346, 33850878, 39094111, 30649555, 35933844, 33046877, 36829404, 39782348, 39116424, 30970066, 39588056, 31027888, 37178815, 34510769, 31308591, 34741612, 35185510, 36933920, 35818405, 37829112, 35231164, 36398287, 37011665, 39228383, 37570047, 35420019, 30981192, 39078385, 38419880, 39773284, 39398664, 32793644, 30914775, 30452062, 36305711, 34547342, 37137062, 34718079, 33864839, 37797416, 31518998, 37724380, 34586919, 31791395, 35047554, 34799425, 37444546, 36572050, 32005584, 36952108, 33916799, 38540505, 34453841, 35082151, 39090789, 33759916, 30538065, 35312825, 31503862, 39128144, 37646076, 30970048, 35257573, 38403469, 33630048, 32144820, 30397772, 39430164, 38059630, 39852709, 34439170, 32428214, 31506984, 33820992, 34681182, 30828310, 32607679, 35787104, 37439651, 31273237, 30125816, 33372982, 36577114, 34846928, 35936406, 34992596, 39036549, 37039117, 36515141, 37177691, 31709884, 38645860, 33416145, 39265132, 38154006, 31985138, 32155550, 38495101, 34130306, 34962545, 39891923, 38752701, 39300411, 31145829, 32663046, 39330400, 34440899, 35803434, 38592059, 33592773, 33365248, 33565926, 39336169, 34028940, 32358967, 37815113, 31276057, 32164954, 33241111, 35475094, 31024873, 38500118, 30089616, 36066057, 30175480, 35218308, 35757748, 34666855, 39631540, 35217373, 31272327, 31452343, 31660060, 34067810, 30589516, 38109304, 34953946, 33048057, 38344257, 35071677, 38320763, 32809464, 35187484, 33663609, 33704047, 33927836, 38081548, 34357675, 34550443, 34297117, 37980262, 36712046, 39844189, 30037325, 30838616, 34836702, 36942555, 32837018, 37400852, 31310328, 31797201, 31571213, 35871970, 31800040, 33863978, 35388967, 32340478, 32856270, 30949612, 37652413, 35771496, 34184100, 35241068, 30289581, 35201523, 37354138, 31092810, 38205268, 33841610, 39271663, 33467361, 30745297, 30167281, 30706474, 39272693, 32382156, 31984618, 35150525, 31033215, 30950523, 35507582, 31271640, 31830836, 32920771, 32057497, 35859413, 39519034, 37644356, 35263802, 35985479, 30958024, 32436717, 30477225, 32731291, 34554611, 37472528, 31753535, 38979936, 32284912, 33736218, 37599746, 34156717, 33821269, 32059124, 31058543, 34613064, 32756784, 32723163, 39216753, 34551650, 31088270, 32603460, 31169007, 33608757, 34755926, 30834507, 35683412, 31796672, 36750933, 31339001, 33240207, 31449537, 32214464, 38663094, 38015685, 39958336, 33739789, 33589252, 32822335, 35216162, 39401861, 30163645, 30708025, 37671492, 39148734, 36684413, 36230109, 34010520, 34460551, 37939543, 30451433, 31853325, 34107647, 36711417, 33812161, 33330682, 32041152, 34971875, 33451500, 39428264, 32465788, 38848356, 38200136, 38064734, 30045078, 33372344, 36357061, 33185188, 34518648, 30602752, 38303429, 30445274, 30354929, 37291291, 31263598, 38859543, 32748004, 34569734, 33029775, 36544336, 33862410, 30097127, 31262149, 35469955, 37686326, 36946789, 32117326, 36174110, 35411905, 37518912, 33859161, 36737999, 31609228, 31888725, 38511747, 35608334, 33443202, 38757849, 35118066, 36580811, 33131667, 39516446, 36431046, 37244481, 34978160, 39005422, 37957986, 33743115, 31631455, 36455003, 31505731, 31540850, 30606623, 30347868, 30551813, 39943078, 36113830, 33903700, 30228689, 30392126, 30852226, 31778525, 35928028, 30865290, 33986047, 38808436, 33678469, 34248775, 32395081, 31494077, 33394581, 35213867, 35727757, 32964221, 36777551, 35911742, 36121419, 34683600, 39107382, 30946405, 33962177, 38886396, 38243263, 35252731, 33186910, 30466822, 35768815, 35289144, 37888531, 36487609, 30826065, 31501334, 38477585, 30343394, 31501159, 38928931, 31958560, 36029370, 31423742, 39381787, 34575919, 39401178, 37729408, 31495997, 36536176, 37403840, 30133203, 34346677, 39135241, 33675851, 38892775, 30034369, 35655254, 35524095, 36869279, 32798963, 36888841, 34452277, 35472547, 38910359, 31883804, 35685118, 31783997, 39793001, 33367754, 35701550, 35760876, 32366341, 35534113, 39577325, 38005497, 30031418, 34204725, 31028994, 33162984, 34386732, 38375026, 31260288, 38032374, 38014082, 35391202, 36449740]

# tids = [32229263]
# tids=[38863565]
# tids = [37674722, 31893008, 36117548, 39769205, 36658829, 39809443, 36645193, 39977184, 34462883, 37624293, 37018118, 33969179]
hids = []

df_con = pd.DataFrame(data = {'time_grid_abs':[0]})
df_con
split_tids = {}
for i, tid in enumerate(tids):
    print(i)
    hid = df_transfers[df_transfers['transfer_id']==tid].iloc[0].hadm_id
    hids.append(hid)

    df2_tr = df_transfers[df_transfers['hadm_id']==hid]
    # df2_tr
    t_in = df2_tr[df2_tr['transfer_id']== tid].intime.values[0]
    t_out = df2_tr[df2_tr['transfer_id']== tid].outtime.values[0]
    # print('t_in, t_out \n ',t_in, t_out)
    # if (t_out-t_in) / np.timedelta64(1,'h') < 24:
    #     tids.remove(tid)
    #     hids.remove(hid)
    #     continue
    df_temp = f_extract(hid, t_in, t_out)
    # df_con.iloc[-1]['time_grid_abs']
    df_temp['time_grid_abs'] = df_temp['time_grid'] + df_con.iloc[-1]['time_grid_abs']
    df_temp['tid'] = tid
    df_temp['hid'] = hid
    df_temp = df_temp.sort_values(by=['tid','time_grid'])
    
    df_con = pd.concat([df_con, df_temp])
    
df_con = df_con.iloc[1: , :]
df_con['value'] = df_con['value'].astype(float)
df_con = df_con.sort_values(by=['tid','label','time_grid'])
df_con['x_grid'] = df_con['tid'].astype(str) + '_' + df_con['time_grid'].astype(str)

df_con



,time_grid_abs
0,0


0


NameError: name 'f_extract' is not defined

In [108]:
q = (df_con['tid']==34187423.0) & (df_con['label']=='Lactate')
(df_con[q].sort_values(by=['label','time_grid']).head(50))


,time_grid_abs,charttime,itemid,value,label,type,time_grid,mask,change,lb,ub,value2,label2,tid,hid,x_grid
446,0.357500,2189-02-19 15:01:00,50813.0,7.3,Lactate,lab,0.357500,H,FH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_0.3575
447,1.740833,2189-02-19 16:24:00,50813.0,8.7,Lactate,lab,1.740833,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_1.7408333333333332
448,3.407500,2189-02-19 18:04:00,50813.0,10.1,Lactate,lab,3.407500,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_3.4075
449,4.657500,2189-02-19 19:19:00,50813.0,12.2,Lactate,lab,4.657500,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_4.6575
450,6.824167,2189-02-19 21:29:00,50813.0,16.1,Lactate,lab,6.824167,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_6.824166666666667
451,8.990833,2189-02-19 23:39:00,50813.0,16.5,Lactate,lab,8.990833,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_8.990833333333333
452,12.357500,2189-02-20 03:01:00,50813.0,12.4,Lactate,lab,12.357500,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_12.3575
453,14.690833,2189-02-20 05:21:00,50813.0,9.8,Lactate,lab,14.690833,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_14.690833333333334
454,18.207500,2189-02-20 08:52:00,50813.0,9.2,Lactate,lab,18.207500,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_18.2075
455,25.990833,2189-02-20 16:39:00,50813.0,6.1,Lactate,lab,25.990833,H,HH,0.5,2.0,NaN,NaN,34187423.0,28735920.0,34187423.0_25.990833333333335


# Visualization

## average occurence

In [109]:
my_list= [
          {'type':'lab', 'col':'type', 'val':['lab'], 'sym':'circle', 'size':6, 'color':'gray', 'name':'Samples-Lab'}, 
          {'type':'lab', 'col':'change', 'val':['NH','FH'], 'sym':'triangle-up', 'size':12, 'color':'red', 'name':'High'},
          {'type':'lab', 'col':'change', 'val':['NL','FL'], 'sym':'triangle-down', 'size':12, 'color':'blue', 'name':'Low'},
          {'type':'lab', 'col':'change', 'val':['LN','HN','FN'], 'sym':'circle', 'size':8, 'color':'green', 'name':'Normal'},   

          
    
    
            {'type':'vital', 'col':'change', 'val':['NH','FH'], 'sym':'triangle-up', 'size':12, 'color':'red', 'name':'High'},
          {'type':'vital', 'col':'change', 'val':['NL','FL'], 'sym':'triangle-down', 'size':12, 'color':'blue', 'name':'Low'},
            {'type':'vital', 'col':'change', 'val':['LN','HN','FN'], 'sym':'circle', 'size':8, 'color':'green', 'name':'Normal'},   

          {'type':'med', 'col':'type', 'val':['med'], 'sym':'square', 'size':8, 'color':'green', 'name':'Medication'},

           

            
            
            ]
# print('# of icu stays: ',len(splits))
df_stat = pd.DataFrame()
for subset in my_list:
    marker=dict(symbol=subset['sym'], size=subset['size'], color=subset['color'])

    q= (df_con.type==subset['type']) & (df_con[subset['col']].isin(subset['val']))
    df_temp = df_con[q]
    df_temp2 = df_temp.groupby('tid').size().reset_index(name='count')
    df_temp2['type'] = subset['type']
    df_temp2['val'] = str(subset['val'])
    
    df_stat = pd.concat([df_stat, df_temp2])
    
    
    print("avg # of events for ",subset['type'],subset['val'],len(df_temp))
fig = px.violin(df_stat, x='type',y='count', color='val',  points=None, box=True)
fig.show()

avg # of events for  lab ['lab'] 312
avg # of events for  lab ['NH', 'FH'] 14
avg # of events for  lab ['NL', 'FL'] 22
avg # of events for  lab ['LN', 'HN', 'FN'] 35
avg # of events for  vital ['NH', 'FH'] 108
avg # of events for  vital ['NL', 'FL'] 29
avg # of events for  vital ['LN', 'HN', 'FN'] 137
avg # of events for  med ['med'] 42


## pattern

In [110]:

q= (df_con['type'].isin(['lab','vital'])) #& (df_con['label']=='Lactate')
df = df_con[q].copy()


df["tid"] = df["tid"].astype(str)
df = df.dropna(subset=['mask'])

splits = df.sort_values(by=['tid','time_grid']).groupby('tid')['time_grid_abs'].first().reset_index()


fig = px.scatter(df, x='time_grid_abs', y='label',color='mask')
for index, row in splits.iterrows():
    # print(row['time_grid_abs'])
    # print(row.tid)
    _ = fig.add_vline(x=row['time_grid_abs'], annotation_text = row['tid'])
_=fig.update_traces(marker=dict(size=8,
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

## events+samples

In [124]:


# q= (df_con.type=='vital') & (df_con.change.isin(['NH','1NL', 'FH','1FL']))
# df_ess_vital = df_con[q]

# q= (df_con.type=='lab') & (df_con.change.isin(['NH','1NL', 'FH','1FL']))
# df_ess_lab = df_con[q]

# q= (df_con.type=='med')
# df_ess_med = df_con[q]

# q= (df_con.type=='lab')
# df_sam_lab = df_con[q]

splits = df.sort_values(by=['tid','time_grid']).groupby('tid')['time_grid_abs'].first().reset_index()


all_labels = [
*df_con[df_con['type']=='vital']['label'].unique().tolist(),
*df_con[df_con['type']=='lab']['label'].unique().tolist(),
*df_con[df_con['type']=='med']['label'].unique().tolist(),
]


for i, item in enumerate(all_labels):
    q= (df_con.label==item)
    df_con.loc[q, 'value2'] = df_con[q].groupby('itemid')['value'].transform(lambda x: (x - x.mean()) / x.std()) + i*4




fig = px.scatter(df_con, x= 'time_grid_abs', y='value2', color='label')
for index, row in splits.iterrows():

    _ = fig.add_vline(x=row['time_grid_abs'], annotation_text = row['tid'])
fig.show()

In [123]:
# plot the data
fig = go.Figure()
df_con = df_con.sort_values(by=['time_grid_abs','itemid'])



my_list= [
      # labs
          
          # samples -> gray
          {'type':'lab', 'col':'type', 'val':['lab'], 'sym':'circle', 'size':6, 'color':'gray', 'name':'Samples-Lab', 'showlegend':True}, 

          # samples -> normality 
          {'type':'lab', 'col':'mask', 'val':['H'], 'sym':'circle', 'size':8, 'color':'red', 'name':'High', 'showlegend':True},
          {'type':'lab', 'col':'mask', 'val':['L'], 'sym':'circle', 'size':8, 'color':'blue', 'name':'Low', 'showlegend':True},
          {'type':'lab', 'col':'mask', 'val':['N'], 'sym':'circle', 'size':8, 'color':'green', 'name':'Normal', 'showlegend':True},   

          # events -> normality 
          {'type':'lab', 'col':'change', 'val':['NH','FH'], 'sym':'triangle-up', 'size':12, 'color':'red', 'name':'High', 'showlegend':True},
          {'type':'lab', 'col':'change', 'val':['NL','FL'], 'sym':'triangle-down', 'size':12, 'color':'blue', 'name':'Low', 'showlegend':True},
          {'type':'lab', 'col':'change', 'val':['LN','HN','FN'], 'sym':'square', 'size':8, 'color':'green', 'name':'Normal', 'showlegend':True},   

          
    
      # vitals
            {'type':'vital', 'col':'change', 'val':['NH','FH'], 'sym':'triangle-up', 'size':12, 'color':'red', 'name':'High', 'showlegend':True},
            {'type':'vital', 'col':'change', 'val':['NL','FL'], 'sym':'triangle-down', 'size':12, 'color':'blue', 'name':'Low', 'showlegend':True},
            {'type':'vital', 'col':'change', 'val':['LN','HN','FN'], 'sym':'square', 'size':8, 'color':'green', 'name':'Normal', 'showlegend':True},   
      # meds
          {'type':'med', 'col':'type', 'val':['med'], 'sym':'square', 'size':8, 'color':'green', 'name':'Medication', 'showlegend':True},

           

            
            
            ]
print('# of icu stays: ',len(splits))

for subset in my_list:
  marker=dict(symbol=subset['sym'], size=subset['size'], color=subset['color'])

  q= (df_con.type==subset['type']) & (df_con[subset['col']].isin(subset['val']))
  df_temp = df_con[q]
  fig = fig.add_trace(go.Scatter(x = df_temp["time_grid_abs"],
                                  y = df_temp["label"], mode = 'markers', marker = marker,
                                  name = subset['name'],
                                  showlegend = subset['showlegend']))
  print("avg # of events for ",subset['type'],subset['val'],len(df_temp)/len(splits))



    
for index, row in splits.iterrows():
    # print(row['time_grid_abs'])
    # print(row.tid)
    _ = fig.add_vline(x=row['time_grid_abs'], annotation_text = row['tid'], annotation_position="top",)


from plotly.graph_objs import Layout

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


# fig.update_layout({'paper_bgcolor' : 'rgba(0,0,0,0)','plot_bgcolor' : 'rgba(0,0,0,0)'})

fig.show()
fig.write_html(ADD_DATA+'temp.html')

# of icu stays:  3
avg # of events for  lab ['lab'] 104.0
avg # of events for  lab ['H'] 25.333333333333332
avg # of events for  lab ['L'] 22.666666666666668
avg # of events for  lab ['N'] 56.0
avg # of events for  lab ['NH', 'FH'] 4.666666666666667
avg # of events for  lab ['NL', 'FL'] 7.333333333333333
avg # of events for  lab ['LN', 'HN', 'FN'] 11.666666666666666
avg # of events for  vital ['NH', 'FH'] 36.0
avg # of events for  vital ['NL', 'FL'] 9.666666666666666
avg # of events for  vital ['LN', 'HN', 'FN'] 45.666666666666664
avg # of events for  med ['med'] 14.0


## interval

In [121]:
df_con = df_con.sort_values(by=['tid','itemid','time_grid']).reset_index(drop=True)
q= (df_con['type']=='lab') & (df_con['label']=='Lactate')
df = df_con[q].copy()
df['diff'] = df.groupby(['tid', 'itemid'], as_index=False)['time_grid'].diff()
df
df["tid"] = df["tid"].astype(str)

fig = px.scatter(df, x='time_grid_abs', y='diff',color='tid')

for index, row in splits.iterrows():
    # print(row['time_grid_abs'])
    # print(row.tid)
    _ = fig.add_vline(x=row['time_grid_abs'], annotation_text = row['tid'])

    
fig.show()

,time_grid_abs,charttime,itemid,value,label,type,mask,time_grid,value2,lb,ub,label2,tid,hid,x_grid,diff
